In [24]:
import pandas as pd
"""
This workbook will create two items:
     1 - It will create a running summary file of the maximum values of ESL exceedances per analysis
     2 - It will create an exclamation mark delimited file that can be read by the program "Create_Exceed_Plotfile.py" to be converted into a plot file.     
User will have to enter the ESL value threshold. See "ESL" variable declaration below.
"""

#TASK 1 - INITIATE THE VARIABLES FOR THIS PROCEDURE
#MAXIFILE = "\\WEFS02\Data\Clients\Buckeye Texas Hub LLC\BUC14868-Alteration, SB1126, Modeling-Permit 106594\AMODEL\01 - Model Output\Naphtha_WorstCase.X01_01"
#            \WEFS02\Data\Clients\Buckeye Texas Hub LLC\BUC14868-Alteration, SB1126, Modeling-Permit 106594\AMODEL - Model Output\Naphtha_WorstCase.X01_01
MAXIFILE = 'Permitted_EO_Refined.X01' #TODO Can we make this a list to cycle through?
Output_Sum_File = 'Permitted_EO_Refined - Exceed Summary.txt'
Output_Max_File = 'Permitted_EO_Refined_1xESL.X01'
ESL = 20



In [25]:
#TASK 2 - READ IN THE AERMOD EXCEEDANCE FILE ("MAXIFILE", "*.X01"). Skip the header rows. Set the column headers to something meaningful.
 
df = pd.read_csv(MAXIFILE,delim_whitespace=True,skiprows=7)
df.columns = ['Avg Time','Source Group','Date','UTME','UTMN','Elev','Hill','Flagpole Height','Conc']

#print(df.head())

In [26]:
#TASK 3 - SORT THE DATASET BY UNIQUE UTME-UTMN PAIRS AND CREATE NEW COLUMN OF COMBINED COORDINATES
df.sort_values(by=['UTME', 'UTMN'],inplace=True) 
#x = df['UTME'].unique() # creates a list of unique UTME values
#y = df['UTMN'].unique() # creates a list of unique UTMN values
df['UTME_UTMN'] = df['UTME'].astype(str)+"_"+df['UTMN'].astype(str)

#print(df.head())

In [27]:
# CREATE NEW COLUMNS FOR 2X, 4X, and 10X within the original dataframe. This will fill with 1s and 0s
df['2xESL']  = df['Conc']>=ESL*2
df['4xESL']  = df['Conc']>=ESL*4
df['10xESL'] = df['Conc']>=ESL*10

print(df.head())

       Avg Time Source Group      Date      UTME       UTMN  Elev  Hill  \
238           1          ALL  16011301  409446.0  3315779.0  3.61  3.61   
239           1          ALL  16011301  409471.0  3315754.0  3.61  3.61   
240           1          ALL  16011301  409471.0  3315779.0  3.64  3.64   
10724         1          ALL  16102223  409471.0  3315779.0  3.64  3.64   
241           1          ALL  16011301  409471.0  3315804.0  3.68  3.68   

       Flagpole Height      Conc           UTME_UTMN  2xESL  4xESL  10xESL  
238                0.0  20.00842  409446.0_3315779.0  False  False   False  
239                0.0  20.81941  409471.0_3315754.0  False  False   False  
240                0.0  21.14043  409471.0_3315779.0  False  False   False  
10724              0.0  20.79348  409471.0_3315779.0  False  False   False  
241                0.0  20.77986  409471.0_3315804.0  False  False   False  


In [28]:
# REPLACE 1s and 0s with TRUE and FALSE
df['2xESL'].replace({True:1,False:0},inplace=True)
df['4xESL'].replace({True:1,False:0},inplace=True)
df['10xESL'].replace({True:1,False:0},inplace=True)

print(df.head())

       Avg Time Source Group      Date      UTME       UTMN  Elev  Hill  \
238           1          ALL  16011301  409446.0  3315779.0  3.61  3.61   
239           1          ALL  16011301  409471.0  3315754.0  3.61  3.61   
240           1          ALL  16011301  409471.0  3315779.0  3.64  3.64   
10724         1          ALL  16102223  409471.0  3315779.0  3.64  3.64   
241           1          ALL  16011301  409471.0  3315804.0  3.68  3.68   

       Flagpole Height      Conc           UTME_UTMN  2xESL 4xESL 10xESL  
238                0.0  20.00842  409446.0_3315779.0      0     0      0  
239                0.0  20.81941  409471.0_3315754.0      0     0      0  
240                0.0  21.14043  409471.0_3315779.0      0     0      0  
10724              0.0  20.79348  409471.0_3315779.0      0     0      0  
241                0.0  20.77986  409471.0_3315804.0      0     0      0  


In [29]:
#TASK 4 - CREATE NEW DATAFRAME 'DFESL'. COUNT THE TOTAL NUMBER OF TIMES THE CONC AT THAT UTME_UTMN EXCEEDS THE ESL LEVEL
dfESL = pd.DataFrame(columns=['UTMs','1xESL','2xESL','4xESL','10xESL','SRCGROUP'])
dfESL['UTMs'] = df['UTME_UTMN'].unique()
dfESL['SRCGROUP'] = 'ALL'   #TODO <====  replace this hardcoded value with the value from df['Source Group'] ******
dfESL.set_index(dfESL['UTMs'],inplace=True)
#dfESL.drop(columns = 'UTMs',inplace=True)

print(dfESL.head())

                                  UTMs 1xESL 2xESL 4xESL 10xESL SRCGROUP
UTMs                                                                    
409446.0_3315779.0  409446.0_3315779.0   NaN   NaN   NaN    NaN      ALL
409471.0_3315754.0  409471.0_3315754.0   NaN   NaN   NaN    NaN      ALL
409471.0_3315779.0  409471.0_3315779.0   NaN   NaN   NaN    NaN      ALL
409471.0_3315804.0  409471.0_3315804.0   NaN   NaN   NaN    NaN      ALL
409471.0_3315829.0  409471.0_3315829.0   NaN   NaN   NaN    NaN      ALL


In [30]:
#TASK 5 - CYCLE THROUGH EACH UTM. CREATE A FILTER TO ONLY LOOK AT THAT INDIVIDUAL VALUE IN THE ORIGINAL DATAFRAME, AND COUNT THE NUMBER OF ENTRIES PER UTM COORDINATE PAIR.

for utm in dfESL['UTMs']:
    #print(utm)
    filt = (df['UTME_UTMN'] == utm)
    #print(df[filt])
    val = df[filt]['Conc'].count()
    #print(val, utm)
    dfESL.loc[utm,'1xESL']  = val
    #print(dfESL.loc[utm,'1xESL'])
    
    dfESL.loc[utm,'2xESL']  = df.loc[filt,'2xESL'].sum()
    dfESL.loc[utm,'4xESL']  = df.loc[filt,'4xESL'].sum()
    dfESL.loc[utm,'10xESL'] = df.loc[filt,'10xESL'].sum()
    

In [31]:
#This reports the max of each ESL-exceeding columns
print('Max # Hours above 1xESL :',dfESL['1xESL'].max())
print('Max # Hours above 2xESL :',dfESL['2xESL'].max())
print('Max # Hours above 4xESL :',dfESL['4xESL'].max())
print('Max # Hours above 10xESL :',dfESL['10xESL'].max())

Max # Hours above 1xESL : 119
Max # Hours above 2xESL : 26
Max # Hours above 4xESL : 0
Max # Hours above 10xESL : 0


In [32]:
#You can use this to report specific UTM locations (such as GLCmax)
GLCMAX = "410332.29_3315865.4"
print('\n*********\nMax # Hours at GLCmax \n*********\n',dfESL.loc[GLCMAX])

GLCNI = "410467.31_3314696.15"
try: 
    print('\n*********\nMax # Hours at GLCni \n*********\n',dfESL.loc[GLCNI])
except KeyError: #there may not be any exceeding concentrations at the GLCni
    pass



*********
Max # Hours at GLCmax 
*********
 UTMs        410332.29_3315865.4
1xESL                       113
2xESL                        21
4xESL                         0
10xESL                        0
SRCGROUP                    ALL
Name: 410332.29_3315865.4, dtype: object


In [33]:
#TASK 6 - WRITE TO A NEW SUMMARY TEXT FILE WITH THE UTMS & ELEVATIONS OF NEW AOI DATAFRAME
# Need to remove the index column at the front and replace with "   DISCCART"
# Need to write in the Receptor Grid acceptable formatting?
# Need to change the delimiter from comma (default) to space character.

with open(Output_Sum_File,"a") as file1:
    file1.write('* * * * * * * * * * * * *\n')
    file1.write('File Name: '+MAXIFILE+'\n')
    file1.write('Max # Hours above 1xESL  :'+dfESL['1xESL'].max().astype(str)+'\n')
    file1.write('Max # Hours above 2xESL  :'+dfESL['2xESL'].max().astype(str)+'\n')
    
    #file1.write('Max # Hours above 4xESL  :'+dfESL['4xESL'].max().astype(str)+'\n')
    file1.write('Max # Hours above 4xESL  :')
    file1.write(str(dfESL['4xESL'].max()))
    file1.write('\n')
    #file1.write('Max # Hours above 4xESL  :'+dfESL['4xESL'].max().astype(str)+'\n') # NOTICE. this only works for "numpy.int64" type. When this value is 0, it comes back as "int" type
    # The following two lines are used when the value is "int" type. Notice the difference between "astype(str)" and "str()"
    file1.write('Max # Hours above 10xESL :')
    file1.write(str(dfESL['10xESL'].max()))
    #print(type(dfESL['2xESL'].max()))  
    #print(type(dfESL['4xESL'].max()))
    #print(type(dfESL['10xESL'].max()))
    file1.write('\n')


In [34]:
#TASK 7 - WRITE TO A NEW EXCEEDANCE FILE IN THE SAME FORMAT AS AERMOD'S "exceedance.plt". THIS IS DIFFERENT FROM AERMOD'S OUTPUT X01 FILE
# Need to remove the index column at the front and replace with "   DISCCART"
# Need to write in the Receptor Grid acceptable formatting?
# Need to change the delimiter from comma (default) to space character.

dfExceed = dfESL.copy()
dfExceed.drop(columns=['2xESL','4xESL','10xESL'],inplace=True)
dfExceed.to_csv(Output_Max_File,index=False,sep='!')
#dfESL['UTM','2xESL','SRCGROUP'].to_csv(Output_Max_File,index=False,sep=' ')



#with open(Output_Max_File,"w") as file1:
#    file1.write('*      X             Y         COUNT-CONC    ZELEV    ZHILL    ZFLAG    AVE     GRP  \n')
#    file1.write('*_____________ _____________ _____________ ________ ________ ________ ______ ________\n')
#    for index, row in df.iteritems():
#        #print('row',row)
#        print(df.loc[row,'UTME'])
#        #print(row['UTME'])
#        break
        
#    for utm in dfESL['UTMs']:
#        break
#        filt = (df['UTME_UTMN'] == utm)
#        print(utm)
#        utme = df.loc[filt]['UTME']
#        print(utme)
#        utmn = df.loc[filt]['UTMN']
#        print(utmn)
#        
#        countconc = df.loc[filt,'2xESL'].astype(str)   # <-- CHANGE THIS LEVEL FRO 2xESL to either 4x or 10x IF YOU WANT A DIFFERENT THRESHOLD
#        print(countconc)
        
#        extra = '     0.00     0.00     0.00 1      '
#        group = df[filt]['Source Group'].astype(str)
#        print(group)
#        #file1.write('   '+utm+' '+str(countconc)+' '+extra+' '+str(group)+'\n')
#        break
#        file1.write('   '+str(utme)+' '+str(utmn)+' '+str(countconc)+' '+extra+' '+str(group)+'\n')

        